# LEO Gens (GTA)

In [1]:
from matrix import *
from context import *
from nets.leo import *
from ds.leo_multi_data import *

np.set_printoptions(suppress=True)

In [2]:
def set_gpu_memory_growth_mode(gpu_id=0):
    import tensorflow as tf
    try:
        gpus = tf.config.experimental.list_physical_devices('GPU')
        tf.config.experimental.set_memory_growth(gpus[gpu_id], True)
    except RuntimeError as e:
        print(e)
set_gpu_memory_growth_mode()        

# Context

In [3]:
config = {
    'seed' : 1234,
    'use_64bits': False,
    'shuffle' : True,
    'zero_kappa': False,
    'use_calib_pick': True,
    'use_valid_only': True,
    'remove_blurry': False,
    'gta_mode': 'kappa_angle',   # 'mixed_effect',
    
    # LEO & target parameter theta dim
    'num_latents': 4,
    # 'gen_theta_dims': 52+4+1+3,
    'gen_theta_dims': 4,     # 128
    'num_k_shots': 5,
    'num_valid_shot': 5,
        
    # Batch & Step size
    'batch_size' : 2,
    'first_decay_steps': 20,
    'meta_lr': 1e-5,
    'theta_lr': 1e-6,
    'latent_lr': 1e-6,
    'num_latent_grad_steps' : 5,
    'num_finetune_grad_steps' : 2,
    
    'gradient_threshold': 0.05,
    'gradient_norm_threshold': 0.05,
 
    # Regularizer Term
    'dropout_rate': 0.4,
    'kl_weight': 5e-1,
    'l2_penalty_weight': 1e+1,
    'encoder_penalty_weight': 1e-4,
    'orthogonality_penalty_weight': 1e-3,
    
    'num_epochs' : 200,
    'npy_root_path': '/home/elvin/host/mnt/ssd3/nps/',
    'landmark_root_path': '/home/elvin/host/mnt/ssd3/lms/',
    
}

ctx = Context.create(config)
set_random_seed(ctx.seed)

In [4]:
LEO_PICK_PIDS = ['00239','00247', '00376', '00623', '00719', '00835', '01231', '01327', '01456', '01460', '01603', '01731', '01734', '01738', '01760', '01762', '01763', '01773', '01786', '01802', '01816', '01817', '01818', '01849', '01860', '01862', '01866', '01869', '01882', '01889', '01907', '01924', '01926', '01930', '01933', '01936', '01961', '01965', '01979', '01984', '02015', '02022', '02023', '02024', '02028', '02038', '02058', '02064', '02077', '02085', '02086', '02092', '02105', '02112', '02114', '02131', '02136', '02152', '02159', '02161', '02168', '02319', '02347', '02358', '02359', '02361', '02367', '02373', '02394', '02420', '02421', '02440', '02459', '02465', '02478', '02518', '02522', '02524', '02575', '02576', '02581', '02585', '02587', '02732', '02763', '02954', '03006', '03214', '03231', '03263']

In [5]:
LEO_TEST = np.random.choice(LEO_PICK_PIDS, 30, replace=False)
LEO_TRAIN = [pid for pid in LEO_PICK_PIDS if pid not in LEO_TEST]

In [6]:
print(LEO_TRAIN)
print(sorted(LEO_TEST))

['00239', '00247', '00376', '00623', '00835', '01231', '01327', '01456', '01731', '01734', '01738', '01760', '01762', '01763', '01773', '01802', '01816', '01818', '01849', '01860', '01862', '01866', '01882', '01907', '01924', '01926', '01933', '01965', '01979', '02023', '02024', '02058', '02064', '02077', '02085', '02086', '02092', '02105', '02112', '02136', '02152', '02159', '02319', '02347', '02367', '02373', '02420', '02421', '02465', '02518', '02522', '02575', '02576', '02581', '02585', '02732', '02954', '03214', '03231', '03263']
['00719', '01460', '01603', '01786', '01817', '01869', '01889', '01930', '01936', '01961', '01984', '02015', '02022', '02028', '02038', '02114', '02131', '02161', '02168', '02358', '02359', '02361', '02394', '02440', '02459', '02478', '02524', '02587', '02763', '03006']


# Gen System

In [7]:
mdp = MultiDataProvider(ctx, LEO_TRAIN)

'__init__'  39.08 s


In [8]:
mat = Matrix(ctx)
net = Leo.create(ctx)

In [9]:
net.train(mdp, mat, num_epochs=500, is_leo=True)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)


KeyboardInterrupt: 

# 평가

In [ ]:
dp = DataProvider(ctx, "01866")

In [ ]:
result = net.evaluate(dp, use_cali_set=True, use_last_gen=False)

In [ ]:
def analysis(res):
    errs = [item.err for item in res]
    df = pd.DataFrame(errs)
    describe = df.apply(lambda x: x.describe([.1,.2,.3,.4,.5,.6,.7,.8,.9]))
    
    pcs = [0]
    for i in range(4, 13):         # 10~90%
        d_row = describe.iloc[i]
        pcs.append(float(d_row.values))
    
    pc_ids = {i:[] for i in range(9)}
    for item in res:
        err = item.err
        diff = item.true - item.pred
        for i in range(9):
            if pcs[i] < err and err <= pcs[i+1]:
                t, p = item.true, item.pred
                pc_ids[i].append((int(item.id), (float(p[0]), float(p[1]))))
    return describe, pc_ids

In [ ]:
d, ids = Matrix.statistics(result, analysis)

*** 
*** 

## DOING
- 일단 전체 데이터셋으로 eyenet 모듈 선학습하기
  - 선훈련 장점: genc 모듈 학습이 안정적이고 전체 과정의 수렴 속도가 빠를 듯
  - 선훈련 단점: eyenet 모듈이 optic 기준으로 학습이 안되고 전체 사람의 평균으로 bias를 학습하게 됨

## Queue
- genc 입력으로 계산 결과 오차 정보도 추가하기
  - kappa 3 mlp로 해보기

- 레퍼런스 모델에서 조금씩 바꿔보며 LEO 적용 실패하는 이유 살펴보기 
- STEP2 range multiply 하기
- theta에서 바로 뽑아낸 상수들도 activation으로 커팅하기
- 너무 말도 안되는 gradient 버려버리기
- 캘리브레이션 셋 선정하기 

- left, right frame two channel single feed forward 작업  
- mini face frame으로 얼굴 각도 피쳐 사용
- 각각 용도로 분리된 genc
- 최종 gaze에서 ordinal loss + PGD (adversarial attack)
- left, right target 계산 결과의 평균이 아닌 lstm NN으로 판정
  
## DONE
- relation 체크하기
  - 샘플간 cross over 반영하도록 수정함
- to visual axis 생성 축 방향확인하기
- to_visual_axis 함수에서 unit vector로 만들어주기
 - 함수 특성상 항상 unit vector가 나오는데 어느 지점에서 바뀐거지?
 - face_R 보정 행렬이 직교 행렬이 아닌듯 -> pitch 각도로만 생성해서 직접 행렬로 만들기
 - face primary position 보정은 pitch만 하도록 반영하기 
 - pred vec이 정규화 되지 않는 이유 체크하기 (face 보정 수정 후 다시 체크할 것)
- 왜 계속 같은 item들만 처리됨? (-> 아님, meta step 반복때문에 그렇게 보인 것)
- loss nan이 떠야만 genc 모듈은 학습이 진행되는 기현상?
  - genc 모듈이 별도 클래스로 했을 때는 nan 때에만 학습됨
  - leo 모듈로 다시 모두 가져오니 학습이 됨
- frame에 대한 blurry 계산 후 해당 프로파일에서 평균 threshold 정해서 커팅하기
  - 01062 205번째 filter idx (968) 번 사례같은 것
  - zscore -1 sigma 아래 잘라버림
- gen_theta none (non calibration) 상황일 때 기본적인 값들로 동일 로직 수행하기
  - kappa a,b = 0, 0, primary position rotmat: identity


## Insight
- 전체적으로 R 회전을 사용하는 것 보다 그냥 eye_patch의 theta, phi를 gaze로 취급하는 것이 더 좋음
 - 이것은 어쩌면 head pose에 대한 샘플들의 분포가 inbalance 해서 그럴 수 있음
- 하지만 미시적 오차에서 R 행렬 적용 여부보다 더 성능에 영향을 주는 건 미세한 early stop 지점임
  - 비벡터 모델의 실험과 동일하게 굉장히 미세한 차이로 10~30mm가 확확 띄는 현상이 보임
  - lr를 낮게 주고 오랜시간 학습하면 위와 같은 들쭉날쭉 현상은 안정화 되는 편 
  
- 만약 피험자 샘플간 정확도의 차이를 유발하는 요소가 kappa와 같은 각도적인 요소라면 LEO로 해당 각을 찾아내는 것으로 해결될 수 있을 것이고, 그것이 아니라 텍스쳐나 안구 외형의 차이라 크다면 LEO 보다는 MAML의 접근이 더 잘 될 것이다. 